# 맛집

## 필요 라이브러리

In [35]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from urllib.request import urlopen, urlretrieve
from urllib.parse import quote_plus
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [36]:
import os
os.getcwd()
os.chdir('C:\\Users\\admin')

In [37]:
driver = webdriver.Chrome('chromedriver.exe')

C:\Users\admin\AppData\Local\Temp\ipykernel_1856\530843676.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


## 맛집 찾기

In [38]:
url = "https://map.naver.com/"
driver.get(url)

In [40]:
location = input('위치를 입력하세요: ')
xpath = '/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div/div/input'
search = driver.find_element(By.XPATH, xpath)
search.clear()
search.send_keys(f'{location} 맛집')
search.send_keys(Keys.ENTER)

위치를 입력하세요:  이대


## 식당 이름 따기

In [41]:
driver.switch_to.frame('searchIframe')

In [42]:
import datetime
import time

def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        scroll = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div[1]')
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll)
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [43]:
doScrollDown(5)

In [44]:
rec = driver.find_elements(By.CLASS_NAME, 'place_bluelink.TYaxT')

li2 = []
for i in range(0, len(rec)):
    li2.append(rec[i].text)

# li2

In [45]:
len(li2)

54

## 전체 텍스트 따기

In [46]:
rec = driver.find_elements(By.CLASS_NAME, 'UEzoS.rTjJo')

lists = []
for i in range(0, len(rec)):
    lists.append(rec[i].text)

# lists

In [47]:
lists[-1]

'전골떡볶이 덕미가 이대본점\nTV찾아라맛있는TV 방영\n이대 맛집 저렴한 즉석 떡볶이\n영업 전\n별점\n4.47방문자리뷰 220블로그리뷰 224\n전골떡볶이 덕미가 이대본점\n전골떡볶이 덕미가 이대본점\n전골떡볶이 덕미가 이대본점\n이미지 더 있음'

In [48]:
len(lists)

54

In [49]:
driver.switch_to.parent_frame()

## \n 없애기

In [50]:
import re

In [51]:
for i in range(0, len(lists)):
    p = re.compile(r'\n')
    lists[i] = p.sub(' ', lists[i])
# lists

## 리뷰 뽑기 + 빈칸 0으로 바꾸기 + 리스트 하나로 만들기

In [52]:
ind = []
for i in range(0, len(lists)):
    a = re.compile(r'\d\.\d{2}')
    ind.append(a.findall(lists[i]))
    for i in range(0, len(ind)):
        if ind[i] == []:
            ind[i] = '0'
            
# reviews1 = [item for sublist in ind for item in sublist][2:22]
# reviews2 = [item for sublist in ind for item in sublist][24:]
# reviews = reviews1 + reviews2
reviews = [item for sublist in ind for item in sublist]

In [53]:
# reviews

## 총 리뷰 수 (방문자 + 블로그)

In [54]:
rvs = []
for i in range(0, len(lists)):
    a1 = re.compile(r'(방문자리뷰)\s(.+)(블로그리뷰)')
    rvs.append(a1.search(lists[i]).group(2))
    for i in range(0, len(rvs)):
        if rvs[i] == []:
            rvs[i] = '0'

In [55]:
rvs[:3]

['221', '347', '380']

In [56]:
rv = []
for i in range(0, len(lists)):
    a1 = re.compile(r'(방문자리뷰)\s(.+)(블로그리뷰)\s(\d+)')
    rv.append(a1.search(lists[i]).group(4))
    for i in range(0, len(rv)):
        if rv[i] == []:
            rv[i] = '0'

In [57]:
rv[:3]

['127', '106', '205']

In [58]:
rvs = [i.replace(',', '') for i in rvs]
rv = [i.replace(',', '') for i in rv]

In [59]:
int(rvs[0]) + int(rv[0])

348

In [60]:
tot = []
for i in range(0, len(rvs)):
    tot.append(int(float(rvs[i])) + int(float(rv[i])))

In [61]:
len(tot)

54

# 리뷰 종류 따기

In [62]:
driver.switch_to.parent_frame()

In [63]:
def ScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        sc = driver.find_element(By.XPATH, '/html')
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", sc)
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [65]:
# url = driver.current_url
# resp = requests.get(url)
from tqdm.notebook import tqdm
import time

driver.switch_to.frame('searchIframe')

e = driver.find_elements(By.CLASS_NAME, 'place_bluelink.TYaxT')

l = []
types = []
for i in tqdm(range(2, 22)):
    e[i].click()
    driver.switch_to.parent_frame()
    driver.switch_to.frame('entryIframe')
    ScrollDown(3)
    com = driver.find_elements(By.CLASS_NAME, 'nWiXa')
    if len(com) == 0:
        l.append('x')
        l.append('x')
        l.append('x')
        l.append('x')
        l.append('x')
    else:
        for j in range(0, len(com)):
            l.append(com[j].text)
    tps = driver.find_element(By.CLASS_NAME, 'DJJvD')
    types.append(tps.text)
    driver.switch_to.parent_frame()
    driver.switch_to.frame('searchIframe')

driver.switch_to.parent_frame()
driver.switch_to.frame('searchIframe')

for i in tqdm(range(24, 54)):
    e[i].click()
    driver.switch_to.parent_frame()
    driver.switch_to.frame('entryIframe')
    ScrollDown(3)
    com = driver.find_elements(By.CLASS_NAME, 'nWiXa')
    if len(com) == 0:
        l.append('x')
        l.append('x')
        l.append('x')
        l.append('x')
        l.append('x')
    else:
        for j in range(0, len(com)):
            l.append(com[j].text)
    tps = driver.find_element(By.CLASS_NAME, 'DJJvD')
    types.append(tps.text)
    driver.switch_to.parent_frame()
    driver.switch_to.frame('searchIframe')

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [66]:
types

['초밥,롤',
 '초밥,롤',
 '다이어트,샐러드',
 '베이글',
 '중식당',
 '베이커리',
 '이탈리아음식',
 '카페',
 '베이커리',
 '인도음식',
 '돈가스',
 '돈가스',
 '태국음식',
 '도시락,컵밥',
 '찌개,전골',
 '떡볶이',
 '아시아음식',
 '베이커리',
 '호떡',
 '카페,디저트',
 '양식',
 '일식당',
 '피자',
 '종합분식',
 '한정식',
 '중식당',
 '한식',
 '카페',
 '중식당',
 '샤브샤브',
 '카페,디저트',
 '한식',
 '양식',
 '맥주,호프',
 '만두',
 '카페,디저트',
 '카페',
 '칼국수,만두',
 '양식',
 '카페,디저트',
 '한식',
 '일식당',
 '칼국수,만두',
 '바(BAR)',
 '해물,생선요리',
 '샌드위치',
 '한식',
 '스파게티,파스타전문',
 '다이어트,샐러드',
 '떡볶이']

In [67]:
len(l)

250

In [68]:
chunks = [l[x:x+5] for x in range(0, len(l), 5)]
len(chunks)

50

## 데이터 프레임 만들기

In [69]:
import pandas as pd

In [70]:
df = pd.DataFrame(zip(li2, reviews, tot), columns=['식당이름', '별점', '리뷰수'])

In [71]:
len(reviews)

54

In [72]:
no1 = df.iloc[2:22, :]
no2 = df.iloc[24:, :]
frames = [no1, no2]
f_df = pd.concat(frames)
f_df.reset_index(drop=True, inplace=True)
f_df[:3]

,식당이름,별점,리뷰수
0,하늘초밥,4.65,585
1,연어초밥,4.67,1487
2,위샐러듀 이대점,4.61,5878


## 위도 경도 불러오기

### 함수

In [73]:
def place_find(s) :
    # 접속 url
    url = 'http://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(s)
    
    # headers 입력
    headers = {
        # 카카오지도 접속
        "Authorization" : "KakaoAK de2ab0f4e5f24304ac7898b7de2cacd8"
    }
    # API 요청
    places = requests.get(url, headers=headers).json()['documents']
    try :
        places = places[0]
        df1_name  = places['place_name']
        위도 = places['y']
        경도 = places['x']
        
        result = [df1_name, 위도, 경도]
        
        return result
    except:
        pass

In [74]:
correct = [] 

for i in range(len(f_df)):
    correct.append(location + " " + f_df.iloc[i].식당이름)
correct_location = pd.DataFrame (correct, columns = ['식당이름'])
# correct_location

In [75]:
# 테스트
place_find(correct_location['식당이름'][0])

['하늘초밥', '37.5573085850893', '126.94669402413']

### 위도 경도 받기

In [76]:
# 위경도 가져오기
food_place_list = []

for i in correct_location['식당이름']:
    data = place_find(i)
    food_place_list.append(data)

In [77]:
food_place_list[:3]

[['하늘초밥', '37.5573085850893', '126.94669402413'],
 ['연어초밥', '37.5570648016113', '126.941972390604'],
 ['위샐러듀 이대점', '37.5594592641306', '126.944732224555']]

In [78]:
# None을 []으로 바꾸기
for i in range(0, len(food_place_list)):
    if food_place_list[i] is None:
        food_place_list[i] = []

### 별점이랑 합치기

In [79]:
for i in range(0, len(food_place_list)):
    if food_place_list[i] == []:
        continue
    else:
        food_place_list[i].append(f_df.iloc[i].별점)
        
food_place_list[:3]

[['하늘초밥', '37.5573085850893', '126.94669402413', '4.65'],
 ['연어초밥', '37.5570648016113', '126.941972390604', '4.67'],
 ['위샐러듀 이대점', '37.5594592641306', '126.944732224555', '4.61']]

### 데이터 프레임 만들기

In [80]:
# 복사
food_place_list_1 = food_place_list

food_place = pd.DataFrame(food_place_list)
# 컬럼 명 지정해주기
food_place.columns = ['식당이름', '위도', '경도', '별점']
food_place['리뷰'] = chunks
food_place['리뷰수'] = tot[2:22] + tot[24:]
food_place[:3]

,식당이름,위도,경도,별점,리뷰,리뷰수
0,하늘초밥,37.5573085850893,126.94669402413,4.65,"[""음식이 맛있어요"", ""재료가 신선해요"", ""가성비가 좋아요"", ""친절해요"", ""...",585
1,연어초밥,37.5570648016113,126.941972390604,4.67,"[""음식이 맛있어요"", ""가성비가 좋아요"", ""친절해요"", ""재료가 신선해요"", ""...",1487
2,위샐러듀 이대점,37.5594592641306,126.944732224555,4.61,"[""음식이 맛있어요"", ""재료가 신선해요"", ""특별한 메뉴가 있어요"", ""인테리어가...",5878


# 리뷰 내용 / 링크 따기

In [81]:
driver.switch_to.parent_frame()

In [82]:
url = driver.current_url
resp = requests.get(url)

driver.switch_to.frame('searchIframe')
e = driver.find_elements(By.CLASS_NAME, 'place_bluelink.TYaxT')

f = []
names = []
names1 = []
all_rvs = []

for i in tqdm(range(2, 22)):
    e[i].click()
    
    # 링크 따기
    url = driver.current_url
    f.append(url)
    
    # 리뷰 내용
    time.sleep(1)
    url = driver.current_url
    driver.switch_to.parent_frame()
    driver.switch_to.frame('entryIframe')
    asdf = driver.find_element(By.CLASS_NAME, 'tpj9w._tab-menu')
    review_link = asdf.get_attribute('href')
    review_link
    driver.switch_to.parent_frame()
    
    p = re.compile(r'(.+/)(home)')
    urla = p.search(review_link).group(1)
    urlb = 'review/visitor'
    url = urla+urlb
    # print(url)
    
    # 리뷰들 불러오기
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser',from_encoding='utf-8')
    rest_name = soup.find(class_= 'Fc1rA').text
    names.append(rest_name)
    # print(rest_name)
    
    reviewss = soup.find_all(class_='zPfVt')
    comments = [j.text for j in reviewss]
    for k in range(0, len(comments)):
        comments[k] = re.sub(r'[^가-힣]', ' ', comments[k])
        names1.append(rest_name)
    all_rvs.append(comments)
    driver.switch_to.parent_frame()
    driver.switch_to.frame('searchIframe')


    
for i in tqdm(range(24, 54)):
    e[i].click()
    
    # 링크 따기
    url = driver.current_url
    f.append(url)
    
    # 리뷰 내용
    time.sleep(1)
    url = driver.current_url
    driver.switch_to.parent_frame()
    driver.switch_to.frame('entryIframe')
    asdf = driver.find_element(By.CLASS_NAME, 'tpj9w._tab-menu')
    review_link = asdf.get_attribute('href')
    review_link
    driver.switch_to.parent_frame()
    
    p = re.compile(r'(.+/)(home)')
    urla = p.search(review_link).group(1)
    urlb = 'review/visitor'
    url = urla+urlb
    # print(url)

    # 리뷰들 불러오기
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser',from_encoding='utf-8')
    rest_name = soup.find(class_= 'Fc1rA').text
    names.append(rest_name)
    # print(rest_name)
    
    reviewss = soup.find_all(class_='zPfVt')
    comments = [j.text for j in reviewss]
    for k in range(0, len(comments)):
        comments[k] = re.sub(r'[^가-힣]', ' ', comments[k])
        names1.append(rest_name)
    all_rvs.append(comments)
    driver.switch_to.parent_frame()
    driver.switch_to.frame('searchIframe')

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [83]:
all_rvs

[['판초밥집에서 히까리를 이렇게 맛있게 내주신다구요  이게 호불호 있을수있는 가격인가 싶은데 무적권 저희동네에 있으면 감사한 수준입니다  가성비란 단어가 실례되지는 않나 싶을 정도로 맛있습니다   주류종류도 매우 다양해서 술쟁이들한테는 너무나도 감사한 곳입니다   너무 맛있고 감사한 하루였습니다',
  '너무너무 맛있어요   이 가격에 이렇게 신선한 초밥을 먹을 수 있어서 행복해요           주변 지인들한테도 소문내고 있어요       우니 먹고싶었는데 없어서 아쉽지만 그래도 아귀간의 참맛을 느끼고 왔습니다    왜 하늘초밥 단골손님들이 아귀간을 찾는지 알게되었어요     사장님 오래오래 장사해주세오  ',
  '하늘초밥은 사랑이에요  먹은 초밥중 일등이니 모두 예약이 가능하면 달려가는걸로해요  저두 연말에 다시한번 가고싶어서 예약을 노리지만 성공하길 바라며     욕심쟁이한에 우니초밥이 안나왓다고 섭섭해하지않기    추가해서 묵어도 절대 돈 안아까움',
  '같은 주 금요일에는 친구랑 방문하고 이날은 남자친구랑 방문했어요  욕심쟁이 우니 먹었고 일주일에 두번 먹으러 가도 안질리네요    이날 일일 알바생 같은 분 말투가 까칠하셔서 기분 상했던 거 말곤 다 만족스러웠어요 ',
  '너무 맛있어서 또 먹고싶어서 뻥 안치고 하루에 한번 사진 보거든요    오늘 또 보는김에 리뷰 쓰러왔어요 단새우 정말 최고   저거 찍을때까진 정신이 있었는데   비록 필름 날아갔지만 진짜 맛있었던건 기억나요   맛있어서 과음함거임   새우장도 비려서 못먹는 내가   저 새우는 비린맛  도없고 걍 달아서 추가주문도 했어요 아 그리고 모든 초밥 비릿만    도 없어요 매우 담백쓰    담달 초 예약 까먹은거 매일 후회해요 ',
  '뭐가 뭔진 모르겠지만 맛있어용   전 익힌 새우를 그렇게까지 좋아하지않아 초새우초밥 먹고 맛있다고 느낀적이 없었는데 여긴 정말 맛있더군요     그리고 약간 그을린 연어도 기가막혔습니다  ',
  '서울와서 유명하다는 초밥집 여러 곳 가봤지만 그 

In [84]:
len(f)

50

In [85]:
food_place['link'] = f
food_place['리뷰내용'] = all_rvs
food_place['종류'] = types

In [86]:
food_place[:3]

,식당이름,위도,경도,별점,리뷰,리뷰수,link,리뷰내용,종류
0,하늘초밥,37.5573085850893,126.94669402413,4.65,"[""음식이 맛있어요"", ""재료가 신선해요"", ""가성비가 좋아요"", ""친절해요"", ""...",585,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[판초밥집에서 히까리를 이렇게 맛있게 내주신다구요 이게 호불호 있을수있는 가격인가...,"초밥,롤"
1,연어초밥,37.5570648016113,126.941972390604,4.67,"[""음식이 맛있어요"", ""가성비가 좋아요"", ""친절해요"", ""재료가 신선해요"", ""...",1487,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[연어초밥집인데 진짜 맛있어요 저는 생연어만 시켰는데 연어불초밥이 불맛 적당한 생...,"초밥,롤"
2,위샐러듀 이대점,37.5594592641306,126.944732224555,4.61,"[""음식이 맛있어요"", ""재료가 신선해요"", ""특별한 메뉴가 있어요"", ""인테리어가...",5878,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[이대에서 근무하는 친구가 찐맛집이라고 추천해서 온 곳인데 맛있어요 그리고 확실...,"다이어트,샐러드"


# 감성 분석

## 데이터 프레임

In [87]:
flat_list = []
for sublist in all_rvs:
    zip(names, all_rvs)
    for item in sublist:
        flat_list.append(item)
        
len(flat_list)

497

In [88]:
len(names1)

497

In [89]:
unzip = pd.DataFrame(zip(names1, flat_list), columns=['names', 'comments'])
unzip

,names,comments
0,하늘초밥,판초밥집에서 히까리를 이렇게 맛있게 내주신다구요 이게 호불호 있을수있는 가격인가 ...
1,하늘초밥,너무너무 맛있어요 이 가격에 이렇게 신선한 초밥을 먹을 수 있어서 행복해요 ...
2,하늘초밥,하늘초밥은 사랑이에요 먹은 초밥중 일등이니 모두 예약이 가능하면 달려가는걸로해요 ...
3,하늘초밥,같은 주 금요일에는 친구랑 방문하고 이날은 남자친구랑 방문했어요 욕심쟁이 우니 먹...
4,하늘초밥,너무 맛있어서 또 먹고싶어서 뻥 안치고 하루에 한번 사진 보거든요 오늘 또 보...
...,...,...
492,전골떡볶이 덕미가 이대본점,해물치즈 정말 맛있습니다 마라떡볶이에 바쪘다가 해물치즈에 감동받았어여
493,전골떡볶이 덕미가 이대본점,떡볶이가 자극적이지않아서 좋아요 깻잎 떡볶이 맛있어요
494,전골떡볶이 덕미가 이대본점,날치알 볶음밥 맛있어요
495,전골떡볶이 덕미가 이대본점,중독성 있는 국물과 먹어도먹어도 바닥을 보이지 않는 떡볶이


In [90]:
unzip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   names     497 non-null    object
 1   comments  497 non-null    object
dtypes: object(2)
memory usage: 7.9+ KB


### 내용 없는거 제거

In [91]:
# comment에 내용이 없는 행 파악
print(len(unzip[unzip['comments'] == '']))

# comment에 내용이 없는 행은 지워서 새 df에 저장
full = unzip[unzip['comments'] != '']
full.reset_index(drop=True, inplace=True)

print(len(full))  # comment가 있는 리뷰의 수 

0
497


In [92]:
full

,names,comments
0,하늘초밥,판초밥집에서 히까리를 이렇게 맛있게 내주신다구요 이게 호불호 있을수있는 가격인가 ...
1,하늘초밥,너무너무 맛있어요 이 가격에 이렇게 신선한 초밥을 먹을 수 있어서 행복해요 ...
2,하늘초밥,하늘초밥은 사랑이에요 먹은 초밥중 일등이니 모두 예약이 가능하면 달려가는걸로해요 ...
3,하늘초밥,같은 주 금요일에는 친구랑 방문하고 이날은 남자친구랑 방문했어요 욕심쟁이 우니 먹...
4,하늘초밥,너무 맛있어서 또 먹고싶어서 뻥 안치고 하루에 한번 사진 보거든요 오늘 또 보...
...,...,...
492,전골떡볶이 덕미가 이대본점,해물치즈 정말 맛있습니다 마라떡볶이에 바쪘다가 해물치즈에 감동받았어여
493,전골떡볶이 덕미가 이대본점,떡볶이가 자극적이지않아서 좋아요 깻잎 떡볶이 맛있어요
494,전골떡볶이 덕미가 이대본점,날치알 볶음밥 맛있어요
495,전골떡볶이 덕미가 이대본점,중독성 있는 국물과 먹어도먹어도 바닥을 보이지 않는 떡볶이


## 감성 분석하기

In [93]:
# pip install vader-multi

In [94]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [95]:
analyzer = SentimentIntensityAnalyzer()

In [96]:
analyzer.polarity_scores(full['comments'][0])

{'neg': 0.037, 'neu': 0.68, 'pos': 0.283, 'compound': 0.9232}

In [97]:
pol = []

for i in tqdm(range(0, len(full['comments']))):
    pol.append(analyzer.polarity_scores(full['comments'][i]))

  0%|          | 0/497 [00:00<?, ?it/s]

In [98]:
pol[0]

{'neg': 0.037, 'neu': 0.68, 'pos': 0.283, 'compound': 0.9232}

In [99]:
q = pd.DataFrame(zip(names1, pol), columns=['names', 'scores'])
q

,names,scores
0,하늘초밥,"{'neg': 0.037, 'neu': 0.68, 'pos': 0.283, 'com..."
1,하늘초밥,"{'neg': 0.0, 'neu': 0.544, 'pos': 0.456, 'comp..."
2,하늘초밥,"{'neg': 0.0, 'neu': 0.771, 'pos': 0.229, 'comp..."
3,하늘초밥,"{'neg': 0.058, 'neu': 0.824, 'pos': 0.118, 'co..."
4,하늘초밥,"{'neg': 0.055, 'neu': 0.781, 'pos': 0.164, 'co..."
...,...,...
492,전골떡볶이 덕미가 이대본점,"{'neg': 0.0, 'neu': 0.718, 'pos': 0.282, 'comp..."
493,전골떡볶이 덕미가 이대본점,"{'neg': 0.129, 'neu': 0.537, 'pos': 0.333, 'co..."
494,전골떡볶이 덕미가 이대본점,"{'neg': 0.0, 'neu': 0.619, 'pos': 0.381, 'comp..."
495,전골떡볶이 덕미가 이대본점,"{'neg': 0.268, 'neu': 0.732, 'pos': 0.0, 'comp..."


In [100]:
(q['names'].value_counts())[0]

10

In [101]:
g = pd.DataFrame(q['names'].value_counts())
g.rename(columns={'names': 'counts'}, inplace=True)
g.reset_index(inplace=True)
g[:3]

,index,counts
0,하늘초밥,10
1,조은집,10
2,모미지식당,10


In [102]:
sums = 0
for i in range(20, 30):
    sums += q['scores'][i]['neu']
    avg = round(sums/10, 4)

sums
avg

0.7526

In [103]:
q['names'].value_counts()

하늘초밥              10
조은집               10
모미지식당             10
waffle it up      10
란주탕슉              10
어바웃샤브 이대점         10
에이테이블             10
도꼭지               10
로드샌드위치            10
탭스터               10
미스터서왕만두           10
호밀밭               10
연희동칼국수 본점         10
희게                10
카페페라              10
공릉닭한마리            10
파파노다이닝            10
김광석신촌칼국수 이대직영점    10
세올                10
스탠바이키친            10
소녀방앗간 이화여대점       10
에그 앤드 스푼레이스       10
굿바이슈가베어           10
연어초밥              10
연희동 라이라이 본점       10
삭 수제튀김 전문점        10
유야케도쿄             10
위샐러듀 이대점          10
마더린러 베이글          10
목란                10
오르랔베이커리           10
비아37              10
티앙팡 오후의 홍차        10
비밀                10
아건                10
사장님돈까스 이화여대점      10
방콕익스프레스 신촌본점      10
아콘스톨              10
고냉지               10
카우떡볶이             10
까이식당              10
원즈오운              10
서울호떡 아현점          10
그릭데이 이대본점         10
쏠페스타 이화여대점        10
진돈부리              10
전골떡볶이 덕미가 이대본점    10
피자클럽         

In [104]:
len(q['names'].value_counts())

50

In [105]:
sums = 0
neg_avgs = []
j = 0
for i in range(0, len(q['names'].value_counts())):
    for j in range(j, j+q['names'].value_counts()[i]):
        sums += q['scores'][j]['neg']
    j+=1
    avg = round(sums/q['names'].value_counts()[i], 4)
    neg_avgs.append(avg)
    # print(avg)
    sums = 0
    avg = 0

# neg_avgs

In [106]:
sums = 0
neu_avgs = []
j = 0
for i in range(0, len(q['names'].value_counts())):
    for j in range(j, j+q['names'].value_counts()[i]):
        sums += q['scores'][j]['neu']
    j+=1
    avg = round(sums/q['names'].value_counts()[i], 4)
    neu_avgs.append(avg)
    sums = 0
    avg = 0

# neu_avgs

In [107]:
sums = 0
pos_avgs = []
j = 0
for i in range(0, len(q['names'].value_counts())):
    for j in range(j, j+q['names'].value_counts()[i]):
        sums += q['scores'][j]['pos']
    j+=1
    avg = round(sums/q['names'].value_counts()[i], 4)
    pos_avgs.append(avg)
    sums = 0
    avg = 0

# pos_avgs

In [108]:
sums = 0
comp_avgs = []
j = 0
for i in range(0, len(q['names'].value_counts())):
    for j in range(j, j+q['names'].value_counts()[i]):
        sums += q['scores'][j]['compound']
    j+=1
    avg = round(sums/q['names'].value_counts()[i], 4)
    comp_avgs.append(avg)
    sums = 0
    avg = 0

# comp_avgs

In [109]:
# pd.DataFrame(zip(names, neg_avgs, neu_avgs, pos_avgs, comp_avgs), columns=['names', 'neg_score', 'neu_score', 'pos_score', 'compd_score'])[:10]

In [110]:
food_place['neg_score'] = neg_avgs
food_place['neu_score'] = neu_avgs
food_place['pos_score'] = pos_avgs
food_place['compd_score'] = comp_avgs
food_place[:3]

,식당이름,위도,경도,별점,리뷰,리뷰수,link,리뷰내용,종류,neg_score,neu_score,pos_score,compd_score
0,하늘초밥,37.5573085850893,126.94669402413,4.65,"[""음식이 맛있어요"", ""재료가 신선해요"", ""가성비가 좋아요"", ""친절해요"", ""...",585,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[판초밥집에서 히까리를 이렇게 맛있게 내주신다구요 이게 호불호 있을수있는 가격인가...,"초밥,롤",0.0356,0.7452,0.2192,0.7601
1,연어초밥,37.5570648016113,126.941972390604,4.67,"[""음식이 맛있어요"", ""가성비가 좋아요"", ""친절해요"", ""재료가 신선해요"", ""...",1487,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[연어초밥집인데 진짜 맛있어요 저는 생연어만 시켰는데 연어불초밥이 불맛 적당한 생...,"초밥,롤",0.0105,0.7087,0.2809,0.8001
2,위샐러듀 이대점,37.5594592641306,126.944732224555,4.61,"[""음식이 맛있어요"", ""재료가 신선해요"", ""특별한 메뉴가 있어요"", ""인테리어가...",5878,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[이대에서 근무하는 친구가 찐맛집이라고 추천해서 온 곳인데 맛있어요 그리고 확실...,"다이어트,샐러드",0.0119,0.7526,0.2354,0.8841


# None값 없애기

In [111]:
food_place.dropna(inplace=True)
food_place.reset_index(drop=True, inplace=True)
food_place.isnull().sum()

식당이름           0
위도             0
경도             0
별점             0
리뷰             0
리뷰수            0
link           0
리뷰내용           0
종류             0
neg_score      0
neu_score      0
pos_score      0
compd_score    0
dtype: int64

## 별점 탑10

In [112]:
food_place.sort_values(by=['별점'], ascending=False, inplace=True)
food_place.reset_index(drop=True, inplace=True)
food_place[:3]

,식당이름,위도,경도,별점,리뷰,리뷰수,link,리뷰내용,종류,neg_score,neu_score,pos_score,compd_score
0,진돈부리,37.5569534052124,126.942456981822,4.84,"[""음식이 맛있어요"", ""친절해요"", ""가성비가 좋아요"", ""재료가 신선해요"", ""...",283,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[신선한 연어와 감칠맛나는 가츠동 요리와 매장 운영에 뚝심 있는 사장님 ...,일식당,0.0087,0.6787,0.3126,0.7487
1,굿바이슈가베어,37.5594402689244,126.944573787447,4.83,"[""음식이 맛있어요"", ""재료가 신선해요"", ""친절해요"", ""혼밥하기 좋아요"", ""...",231,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[로지클라우드 소스 마요네즈로 변경했는데 잘 어울리고 맛있었음 직원분 너무 친절하...,"다이어트,샐러드",0.0041,0.7942,0.2017,0.5887
2,탭스터,37.5686427225759,126.93190290238,4.83,"[""음식이 맛있어요"", ""술이 다양해요"", ""친절해요"", ""매장이 청결해요"", ""특...",292,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,"[직원분들께서 넘 친절하시구 피자랑 맥주 맛있어요 , 맥주를 정말 좋아하...","맥주,호프",0.0000,0.6808,0.3192,0.6910


In [113]:
upper = food_place.iloc[:10, :]
upper

,식당이름,위도,경도,별점,리뷰,리뷰수,link,리뷰내용,종류,neg_score,neu_score,pos_score,compd_score
0,진돈부리,37.5569534052124,126.942456981822,4.84,"[""음식이 맛있어요"", ""친절해요"", ""가성비가 좋아요"", ""재료가 신선해요"", ""...",283,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[신선한 연어와 감칠맛나는 가츠동 요리와 매장 운영에 뚝심 있는 사장님 ...,일식당,0.0087,0.6787,0.3126,0.7487
1,굿바이슈가베어,37.5594402689244,126.944573787447,4.83,"[""음식이 맛있어요"", ""재료가 신선해요"", ""친절해요"", ""혼밥하기 좋아요"", ""...",231,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[로지클라우드 소스 마요네즈로 변경했는데 잘 어울리고 맛있었음 직원분 너무 친절하...,"다이어트,샐러드",0.0041,0.7942,0.2017,0.5887
2,탭스터,37.5686427225759,126.93190290238,4.83,"[""음식이 맛있어요"", ""술이 다양해요"", ""친절해요"", ""매장이 청결해요"", ""특...",292,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,"[직원분들께서 넘 친절하시구 피자랑 맥주 맛있어요 , 맥주를 정말 좋아하...","맥주,호프",0.0000,0.6808,0.3192,0.6910
3,아건 이대점,37.55906291657092,126.94492718486593,4.82,"[""음식이 맛있어요"", ""친절해요"", ""가성비가 좋아요"", ""특별한 메뉴가 있어요""...",2148,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[아이들과 이대 나들이와서 특별한 음식 먹고 싶었어요 성공이네요 주차 홍조주차장...,인도음식,0.0097,0.8006,0.1898,0.7496
4,모미지식당,37.55883015675004,126.94428223894934,4.74,"[""음식이 맛있어요"", ""친절해요"", ""특별한 메뉴가 있어요"", ""혼밥하기 좋아요""...",532,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,"[혼밥석 있어서 좋았어요 가지덮밥이 마감이라 육회덮밥 먹었는데 맛있었어요 , 고등어...",한식,0.0268,0.7119,0.2613,0.5960
5,연어초밥,37.5570648016113,126.941972390604,4.67,"[""음식이 맛있어요"", ""가성비가 좋아요"", ""친절해요"", ""재료가 신선해요"", ""...",1487,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[연어초밥집인데 진짜 맛있어요 저는 생연어만 시켰는데 연어불초밥이 불맛 적당한 생...,"초밥,롤",0.0105,0.7087,0.2809,0.8001
6,비아37,37.5589904689193,126.944144041557,4.67,"[""음식이 맛있어요"", ""인테리어가 멋져요"", ""친절해요"", ""특별한 메뉴가 있어요...",907,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[진짜 존맛탱구리에요 샐러드는 별로 기대 안했는데 버섯의 촉촉함과 풍미가 진짜 대...,이탈리아음식,0.0088,0.7154,0.2758,0.8089
7,희게,37.568663448725,126.931908543177,4.66,"[""음식이 맛있어요"", ""친절해요"", ""인테리어가 멋져요"", ""재료가 신선해요"", ...",565,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[잠봉뵈르 가격 원입니다 저처럼 헤매는 분이 있을까 미리 말씀드려요 ...,양식,0.0086,0.7534,0.2379,0.7005
8,하늘초밥,37.5573085850893,126.94669402413,4.65,"[""음식이 맛있어요"", ""재료가 신선해요"", ""가성비가 좋아요"", ""친절해요"", ""...",585,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[판초밥집에서 히까리를 이렇게 맛있게 내주신다구요 이게 호불호 있을수있는 가격인가...,"초밥,롤",0.0356,0.7452,0.2192,0.7601
9,와플잇업 이대2호점,37.5582476571318,126.943325190362,4.63,"[""디저트가 맛있어요"", ""커피가 맛있어요"", ""인테리어가 멋져요"", ""친절해요"",...",1168,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[유명한 와플집이어서 찾아갔어요 신촌 특유의 감성이 느껴지는 분위기에 매장 내외부...,카페,0.0243,0.7139,0.2618,0.5975


## 리뷰수 탑10

In [114]:
food_place.sort_values(by=['리뷰수'], ascending=False, inplace=True)
food_place.reset_index(drop=True, inplace=True)
food_place[:3]

,식당이름,위도,경도,별점,리뷰,리뷰수,link,리뷰내용,종류,neg_score,neu_score,pos_score,compd_score
0,위샐러듀 이대점,37.5594592641306,126.944732224555,4.61,"[""음식이 맛있어요"", ""재료가 신선해요"", ""특별한 메뉴가 있어요"", ""인테리어가...",5878,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[이대에서 근무하는 친구가 찐맛집이라고 추천해서 온 곳인데 맛있어요 그리고 확실...,"다이어트,샐러드",0.0119,0.7526,0.2354,0.8841
1,연희동칼국수 본점,37.5682239892832,126.930748718394,4.32,"[""음식이 맛있어요"", ""주차하기 편해요"", ""친절해요"", ""양이 많아요"", ""재료...",3944,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[쌀쌀한 날 뜨 끈한 사골국물 칼국수 어떠신가요 김치가 일품입니다 점심에 ...,"칼국수,만두",0.0132,0.7973,0.1894,0.5963
2,마더린러베이글,37.558309895691,126.945384976896,0,"[""빵이 맛있어요"", ""특별한 메뉴가 있어요"", ""친절해요"", ""매장이 청결해요"",...",2244,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[첫주문 어떻게 해야할지 모르겠어서 좀 헤맸는데 맛있었습니다 비엘티랑 에그치즈 샌...,베이글,0.0075,0.7896,0.2030,0.6966


In [115]:
rv_upper = food_place.iloc[:10, :]
rv_upper

,식당이름,위도,경도,별점,리뷰,리뷰수,link,리뷰내용,종류,neg_score,neu_score,pos_score,compd_score
0,위샐러듀 이대점,37.5594592641306,126.944732224555,4.61,"[""음식이 맛있어요"", ""재료가 신선해요"", ""특별한 메뉴가 있어요"", ""인테리어가...",5878,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[이대에서 근무하는 친구가 찐맛집이라고 추천해서 온 곳인데 맛있어요 그리고 확실...,"다이어트,샐러드",0.0119,0.7526,0.2354,0.8841
1,연희동칼국수 본점,37.5682239892832,126.930748718394,4.32,"[""음식이 맛있어요"", ""주차하기 편해요"", ""친절해요"", ""양이 많아요"", ""재료...",3944,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[쌀쌀한 날 뜨 끈한 사골국물 칼국수 어떠신가요 김치가 일품입니다 점심에 ...,"칼국수,만두",0.0132,0.7973,0.1894,0.5963
2,마더린러베이글,37.558309895691,126.945384976896,0,"[""빵이 맛있어요"", ""특별한 메뉴가 있어요"", ""친절해요"", ""매장이 청결해요"",...",2244,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[첫주문 어떻게 해야할지 모르겠어서 좀 헤맸는데 맛있었습니다 비엘티랑 에그치즈 샌...,베이글,0.0075,0.7896,0.2030,0.6966
3,아건 이대점,37.55906291657092,126.94492718486593,4.82,"[""음식이 맛있어요"", ""친절해요"", ""가성비가 좋아요"", ""특별한 메뉴가 있어요""...",2148,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[아이들과 이대 나들이와서 특별한 음식 먹고 싶었어요 성공이네요 주차 홍조주차장...,인도음식,0.0097,0.8006,0.1898,0.7496
4,목란,37.56838240474039,126.93047690875805,0,"[""음식이 맛있어요"", ""친절해요"", ""특별한 메뉴가 있어요"", ""재료가 신선해요""...",2109,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[기대를 엄청 한 거에 비해 아쉽긴 해요 코스요 가격대가 저렴한데 대신 양...,중식당,0.0296,0.7813,0.1890,0.6630
5,호밀밭,37.558918423412536,126.94052464390788,4.62,"[""디저트가 맛있어요"", ""특별한 메뉴가 있어요"", ""가성비가 좋아요"", ""친절해요...",1965,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[소문듣고 찾아갔는데 역시 신촌 최고 빙수집 다른 곳 팥은 너무 달기만 한데 ...,카페,0.0058,0.7788,0.2154,0.6433
6,비밀,37.5585538958815,126.943116710815,4.41,"[""빵이 맛있어요"", ""특별한 메뉴가 있어요"", ""친절해요"", ""매장이 청결해요"",...",1624,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[여긴 치아바타 맛집 사진은 잠봉뵈르 치아바타를 이용한 샌드위치가 시그니처...,베이커리,0.0073,0.7012,0.2916,0.5306
7,방콕익스프레스 신촌본점,37.557169356192,126.941868300831,4.36,"[""음식이 맛있어요"", ""양이 많아요"", ""가성비가 좋아요"", ""특별한 메뉴가 있어...",1519,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[똠양꿍은 적당히 칼칼했고 향이 막 강하지 않아서 좋았어요 뿌팟퐁커리에 토핑으로 ...,태국음식,0.0300,0.7436,0.2264,0.5119
8,란주탕슉,37.5581661536053,126.94629502857,4.41,"[""음식이 맛있어요"", ""특별한 메뉴가 있어요"", ""양이 많아요"", ""친절해요"", ...",1517,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[사천꿔바로우 존맛 마늘후레이크와 고추가 적당히 매콤하여 입맛을 돋우어주는 꿔바로우...,중식당,0.0100,0.7326,0.2573,0.5567
9,연어초밥,37.5570648016113,126.941972390604,4.67,"[""음식이 맛있어요"", ""가성비가 좋아요"", ""친절해요"", ""재료가 신선해요"", ""...",1487,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[연어초밥집인데 진짜 맛있어요 저는 생연어만 시켰는데 연어불초밥이 불맛 적당한 생...,"초밥,롤",0.0105,0.7087,0.2809,0.8001


## 감성 분석 탑10

In [116]:
food_place.sort_values(by=['compd_score'], ascending=False, inplace=True)
food_place.reset_index(drop=True, inplace=True)
food_place[-1:]

,식당이름,위도,경도,별점,리뷰,리뷰수,link,리뷰내용,종류,neg_score,neu_score,pos_score,compd_score
42,조은집,37.568571978666405,126.93109929511584,4.24,"[""음식이 맛있어요"", ""가성비가 좋아요"", ""친절해요"", ""양이 많아요"", ""반찬...",969,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[추억 여행으로 몇년만에 방문했어요 계란찜 잡채는 따끈하게 바로 해 주셔서 넘 ...,한정식,0.1024,0.7233,0.1743,0.299


In [117]:
senti10 = food_place.iloc[:10, :]
senti10

,식당이름,위도,경도,별점,리뷰,리뷰수,link,리뷰내용,종류,neg_score,neu_score,pos_score,compd_score
0,티앙팡 오후에 홍차 1호점,37.55841652340589,126.94605943897574,4.51,"[""음료가 맛있어요"", ""디저트가 맛있어요"", ""특별한 메뉴가 있어요"", ""인테리어...",849,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[인테리어도 너무 재미있고 채광이 들어오는 자리도 힐링이더라고요 차를 좋아하시...,카페,0.0000,0.7305,0.2695,0.8937
1,위샐러듀 이대점,37.5594592641306,126.944732224555,4.61,"[""음식이 맛있어요"", ""재료가 신선해요"", ""특별한 메뉴가 있어요"", ""인테리어가...",5878,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[이대에서 근무하는 친구가 찐맛집이라고 추천해서 온 곳인데 맛있어요 그리고 확실...,"다이어트,샐러드",0.0119,0.7526,0.2354,0.8841
2,비아37,37.5589904689193,126.944144041557,4.67,"[""음식이 맛있어요"", ""인테리어가 멋져요"", ""친절해요"", ""특별한 메뉴가 있어요...",907,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[진짜 존맛탱구리에요 샐러드는 별로 기대 안했는데 버섯의 촉촉함과 풍미가 진짜 대...,이탈리아음식,0.0088,0.7154,0.2758,0.8089
3,연어초밥,37.5570648016113,126.941972390604,4.67,"[""음식이 맛있어요"", ""가성비가 좋아요"", ""친절해요"", ""재료가 신선해요"", ""...",1487,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[연어초밥집인데 진짜 맛있어요 저는 생연어만 시켰는데 연어불초밥이 불맛 적당한 생...,"초밥,롤",0.0105,0.7087,0.2809,0.8001
4,오르랔베이커리,37.5574463401982,126.952816767662,4.58,"[""빵이 맛있어요"", ""특별한 메뉴가 있어요"", ""친절해요"", ""인테리어가 멋져요""...",1245,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[드디어 와보는 오르랔 기대만큼 맛있었어요 저는 옥수수 딸기 치킨크림 순으로 ...,베이커리,0.0044,0.6662,0.3294,0.7862
5,하늘초밥,37.5573085850893,126.94669402413,4.65,"[""음식이 맛있어요"", ""재료가 신선해요"", ""가성비가 좋아요"", ""친절해요"", ""...",585,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[판초밥집에서 히까리를 이렇게 맛있게 내주신다구요 이게 호불호 있을수있는 가격인가...,"초밥,롤",0.0356,0.7452,0.2192,0.7601
6,아건 이대점,37.55906291657092,126.94492718486593,4.82,"[""음식이 맛있어요"", ""친절해요"", ""가성비가 좋아요"", ""특별한 메뉴가 있어요""...",2148,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[아이들과 이대 나들이와서 특별한 음식 먹고 싶었어요 성공이네요 주차 홍조주차장...,인도음식,0.0097,0.8006,0.1898,0.7496
7,진돈부리,37.5569534052124,126.942456981822,4.84,"[""음식이 맛있어요"", ""친절해요"", ""가성비가 좋아요"", ""재료가 신선해요"", ""...",283,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[신선한 연어와 감칠맛나는 가츠동 요리와 매장 운영에 뚝심 있는 사장님 ...,일식당,0.0087,0.6787,0.3126,0.7487
8,아콘스톨,37.5583787734025,126.942437777275,0,"[""음식이 맛있어요"", ""가성비가 좋아요"", ""친절해요"", ""양이 많아요"", ""혼밥...",488,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[사장님 진짜 왕왕 친절하시고 김밥이랑 와사비샌드 정말 맛있어요 제가 주먹밥이...,"도시락,컵밥",0.0174,0.7183,0.2643,0.7487
9,스탠바이키친,37.56456065275251,126.94461753264709,4.48,"[""음식이 맛있어요"", ""재료가 신선해요"", ""혼밥하기 좋아요"", ""특별한 메뉴가 ...",1129,https://map.naver.com/v5/search/%EC%9D%B4%EB%8...,[연대 외국어학당과 외국인 기숙사가 가까워선지 유독 외국 친구들이 많이 이용하는 샌...,샌드위치,0.0243,0.6612,0.3146,0.7373


# 지도

### from website

In [118]:
import requests
import numpy as np
import folium
from folium.plugins import MiniMap

## 최종 지도

In [119]:
def make_map(df1, df2, df3):
    # 지도 생성하기
    m = folium.Map(location=upper.iloc[0][1:3],   # 기준좌표: 서울시청
                   zoom_start=15)

    # 미니맵 추가하기
    minimap = MiniMap() 
    m.add_child(minimap)

    # 마커 추가하기
    for i in range(len(df1)):
        popup_text = f'''<a href={upper.iloc[i].link} target='_blank'>
                    {upper.iloc[i]['식당이름']}</a>, {upper.iloc[i]['종류']}<br>
                    별점: {upper.iloc[i]['별점']}<br>'''

        #팝업 크기 조절
        popup = folium.Popup(popup_text, min_width=100, max_width=300)
    
        # 마커 생성
        folium.Marker([upper['위도'][i],upper['경도'][i]],
                  tooltip=upper['식당이름'][i],
                  popup=popup, 
                      icon=folium.Icon(color='red', icon='cutlery')).add_to(m)
        
    for i in range(len(df2)):
        popup_text = f'''<a href={rv_upper.iloc[i].link} target='_blank'>
                    {rv_upper.iloc[i]['식당이름']}</a>, {rv_upper.iloc[i]['종류']}<br>
                    리뷰 수: {rv_upper.iloc[i]['리뷰수']}<br>
                    리뷰: {rv_upper.iloc[i]['리뷰']}<br>'''

        #팝업 크기 조절
        popup = folium.Popup(popup_text, min_width=100, max_width=300)
        
        if (rv_upper['식당이름'][i] == upper['식당이름']).any() == True:
            
            popup_text = f'''<a href={senti10.iloc[i].link} target='_blank'>
            {senti10.iloc[i]['식당이름']}</a>, {senti10.iloc[i]['종류']}<br>
            별점: {upper.iloc[i]['별점']}<br> 
            리뷰 수: {rv_upper.iloc[i]['리뷰수']}<br>'''

            #팝업 크기 조절
            popup = folium.Popup(popup_text, min_width=100, max_width=300)

            # 마커 생성
            folium.Marker([rv_upper['위도'][i],rv_upper['경도'][i]],
                      tooltip=rv_upper['식당이름'][i],
                      popup=popup, 
                          icon=folium.Icon(color='purple', icon='cutlery')).add_to(m)
        else:
            # 마커 생성
            folium.Marker([rv_upper['위도'][i],rv_upper['경도'][i]],
                      tooltip=rv_upper['식당이름'][i],
                      popup=popup, 
                          icon=folium.Icon(color='blue', icon='cutlery')).add_to(m)
    
    for i in range(len(df3)):
        popup_text = f'''<a href={senti10.iloc[i].link} target='_blank'>
                    {senti10.iloc[i]['식당이름']}</a>, {senti10.iloc[i]['종류']}<br>
                    복합 감정 지수: {senti10.iloc[i]['compd_score']}<br>'''

        #팝업 크기 조절
        popup = folium.Popup(popup_text, min_width=100, max_width=300)
        
        if (senti10['식당이름'][i] == upper['식당이름']).any() == True: 
            
            if (senti10['식당이름'][i] == rv_upper['식당이름']).any() == True:
                
                popup_text = f'''<a href={senti10.iloc[i].link} target='_blank'>
                            {senti10.iloc[i]['식당이름']}</a>, {senti10.iloc[i]['종류']}<br>
                            별점: {upper.iloc[i]['별점']}<br> 
                            리뷰 수: {rv_upper.iloc[i]['리뷰수']}<br>
                            복합 감정 지수: {senti10.iloc[i]['compd_score']}<br>'''

                #팝업 크기 조절
                popup = folium.Popup(popup_text, min_width=100, max_width=300)
                
                # 마커 생성
                folium.Marker([senti10['위도'][i],senti10['경도'][i]],
                      tooltip=senti10['식당이름'][i],
                      popup=popup, 
                          icon=folium.Icon(color='green', icon='cutlery')).add_to(m)
            else:
                popup_text = f'''<a href={senti10.iloc[i].link} target='_blank'>
                            {senti10.iloc[i]['식당이름']}</a>, {senti10.iloc[i]['종류']}<br>
                            별점: {upper.iloc[i]['별점']}<br> 
                            복합 감정 지수: {senti10.iloc[i]['compd_score']}<br>'''

                #팝업 크기 조절
                popup = folium.Popup(popup_text, min_width=100, max_width=300)
                
                # 마커 생성
                folium.Marker([senti10['위도'][i],senti10['경도'][i]],
                          tooltip=senti10['식당이름'][i],
                          popup=popup, 
                              icon=folium.Icon(color='pink', icon='cutlery')).add_to(m)

        elif (senti10['식당이름'][i] == rv_upper['식당이름']).any() == True:
            popup_text = f'''<a href={senti10.iloc[i].link} target='_blank'>
                        {senti10.iloc[i]['식당이름']}</a>, {senti10.iloc[i]['종류']}<br>
                        리뷰 수: {rv_upper.iloc[i]['리뷰수']}<br> 
                        복합 감정 지수: {senti10.iloc[i]['compd_score']}<br>'''

            #팝업 크기 조절
            popup = folium.Popup(popup_text, min_width=100, max_width=300)

            # 마커 생성
            folium.Marker([senti10['위도'][i],senti10['경도'][i]],
                      tooltip=senti10['식당이름'][i],
                      popup=popup, 
                          icon=folium.Icon(color='darkpurple', icon='cutlery')).add_to(m)       

        else:
            # 마커 생성
            folium.Marker([senti10['위도'][i],senti10['경도'][i]],
                      tooltip=senti10['식당이름'][i],
                      popup=popup, 
                          icon=folium.Icon(color='orange', icon='cutlery')).add_to(m)
    
    return m

In [120]:
make_map(upper, rv_upper, senti10)

# 참고용

### 맛집 하나

In [121]:
driver.current_url

'https://map.naver.com/v5/search/%EC%9D%B4%EB%8C%80%20%EB%A7%9B%EC%A7%91/place/37792625?c=14129560.0778385,4517378.7134694,14,0,0,0,dh&placePath=%2Fphoto%3Fentry%253Dbmp'

In [122]:
driver.switch_to.parent_frame()

In [123]:
driver.switch_to.frame('searchIframe')

In [124]:
driver.switch_to.parent_frame()

In [125]:
driver.switch_to.frame('entryIframe')

In [126]:
asdf = driver.find_element(By.CLASS_NAME, 'tpj9w._tab-menu')
review_link = asdf.get_attribute('href')
review_link

'https://pcmap.place.naver.com/restaurant/37792625/home'

In [127]:
p = re.compile(r'(.+/)(home)')
p.search(review_link).group(1)

'https://pcmap.place.naver.com/restaurant/37792625/'

In [128]:
p = re.compile(r'(.+/)(home)')

urla = p.search(review_link).group(1)
urlb = 'review/visitor'
url = urla+urlb

In [129]:
url

'https://pcmap.place.naver.com/restaurant/37792625/review/visitor'

In [130]:
import requests

In [131]:
html = requests.get(url)
# html

In [132]:
soup = BeautifulSoup(html.content, 'html.parser',from_encoding='utf-8')
# soup

In [133]:
soup.find(class_= 'Fc1rA').text

'위샐러듀 이대점'

In [134]:
reviewss = soup.find_all(class_='zPfVt')
# reviewss

In [135]:
comments = [i.text for i in reviewss]
comments

['이대에서 근무하는 친구가 찐맛집이라고 추천해서 온 곳인데 맛있어요 ! 그리고 확실히 비건식이라서 다 먹고도 속이 더부룩하지 않아서 좋더라구요! 개인적으로 저 수프같이 생긴게 훨씬 맛있었어요! 빵추가 하고 싶은 맛!!',
 '전부터 오고싶었던 곳인데 근처 들를일이 있어서 방문했어요! 고민하다가 비건메뉴(노바)를 골랐는데 양도 많고 여러가지 건강한 재료들이 많이들어가서 좋았어요! 평소에 잘 접해보지 못한 음식이라 더 맘에들어요 분위기도 아늑하고 혼밥하기 좋아요 🥰',
 '둘이 와서 이대쪽 또 언제 오겠냐며 메뉴 3개 시켰는데 개나댔어요.. 양 진짜 많아서 ㅎ 근데 하나같이 다 맛있어요..ㅠ 루나랑 뽀빠이, 훔스 먹었는데 뽀빠이가 젤 맛있었고 전 루나가 그 다음.. 친구는 훔스가 그 다음으로 맛있다고 했어요!! 결론은 다 맛있다.. 담엔 다른것들도 다 먹어보고 싶어요. 후무스도 맛있고 두부튀김 역시 짱~~!! 자몽이 생각보다 잘 어울려서 또 좋았어요. 한시간 거리지만 재방문.. 아니 삼방문 아 모르겠고 오래오래 이대 앞에 있어줬음 좋겠어요. 사장님(여자시겠죠? 제발) 부자되세요 ',
 '레드풀 스튜가 청국장 같아 징거나 풀문에 나온 밥에도 잘 어울려요. 징거 새우 오징어 신선한 야채들이 맛나요. 풀문 두부튀김 담백하니 좋네요. 가벼운듯하지만 먹다보면 배부르네요. 호밀빠은 구워 나와서 바삭하니 좋고 공갈빵 같은 빵은 쫄깃하니 맛있고 음식과 잘 맞아요~^^',
 '양이 푸짐~ 해서 배불리 먹을 수 있고 \n굴라쉬들은 약간 몸이 허할때 먹음 좋은 보양식 느낌이였어요~ 저는 뽀빠이가 입맛에 가장 맞았고 빵과 함께 먹는게 너무 잘 어울렸어요^^ \n전체적으로 매콤한 편인 줄 모르고 아이를 데려왔는데~ 그부분에대한 코멘트나 선택사항이 있음 좋을 것 같네요^^',
 '너무추워서따뜻한거시켰어요. 찾기가약간힘든데 \n아담하니 혼밥하기좋아요. 이대생들게따뜻한점심이될것같아요.\n샥슈카는 스프러버여서맛있게먹었어요.다음에는뽀빠이를먹어보고싶어요.',
 '가격이 최근에 많이 올랐지만, 역

In [136]:
comments[0].strip(r'[^가-힣]')

'이대에서 근무하는 친구가 찐맛집이라고 추천해서 온 곳인데 맛있어요 ! 그리고 확실히 비건식이라서 다 먹고도 속이 더부룩하지 않아서 좋더라구요! 개인적으로 저 수프같이 생긴게 훨씬 맛있었어요! 빵추가 하고 싶은 맛!!'

In [137]:
re.sub(r'[^가-힣]', ' ', comments[0])

'이대에서 근무하는 친구가 찐맛집이라고 추천해서 온 곳인데 맛있어요   그리고 확실히 비건식이라서 다 먹고도 속이 더부룩하지 않아서 좋더라구요  개인적으로 저 수프같이 생긴게 훨씬 맛있었어요  빵추가 하고 싶은 맛  '

In [138]:
# p = re.compile(r'[가-힣]+')
# p.findall(comments[0])
for i in range(0, len(comments)):
    comments[i] = re.sub(r'[^가-힣]', ' ', comments[i])

In [139]:
comments

['이대에서 근무하는 친구가 찐맛집이라고 추천해서 온 곳인데 맛있어요   그리고 확실히 비건식이라서 다 먹고도 속이 더부룩하지 않아서 좋더라구요  개인적으로 저 수프같이 생긴게 훨씬 맛있었어요  빵추가 하고 싶은 맛  ',
 '전부터 오고싶었던 곳인데 근처 들를일이 있어서 방문했어요  고민하다가 비건메뉴 노바 를 골랐는데 양도 많고 여러가지 건강한 재료들이 많이들어가서 좋았어요  평소에 잘 접해보지 못한 음식이라 더 맘에들어요 분위기도 아늑하고 혼밥하기 좋아요  ',
 '둘이 와서 이대쪽 또 언제 오겠냐며 메뉴  개 시켰는데 개나댔어요   양 진짜 많아서   근데 하나같이 다 맛있어요    루나랑 뽀빠이  훔스 먹었는데 뽀빠이가 젤 맛있었고 전 루나가 그 다음   친구는 훔스가 그 다음으로 맛있다고 했어요   결론은 다 맛있다   담엔 다른것들도 다 먹어보고 싶어요  후무스도 맛있고 두부튀김 역시 짱     자몽이 생각보다 잘 어울려서 또 좋았어요  한시간 거리지만 재방문   아니 삼방문 아 모르겠고 오래오래 이대 앞에 있어줬음 좋겠어요  사장님 여자시겠죠  제발  부자되세요 ',
 '레드풀 스튜가 청국장 같아 징거나 풀문에 나온 밥에도 잘 어울려요  징거 새우 오징어 신선한 야채들이 맛나요  풀문 두부튀김 담백하니 좋네요  가벼운듯하지만 먹다보면 배부르네요  호밀빠은 구워 나와서 바삭하니 좋고 공갈빵 같은 빵은 쫄깃하니 맛있고 음식과 잘 맞아요   ',
 '양이 푸짐  해서 배불리 먹을 수 있고  굴라쉬들은 약간 몸이 허할때 먹음 좋은 보양식 느낌이였어요  저는 뽀빠이가 입맛에 가장 맞았고 빵과 함께 먹는게 너무 잘 어울렸어요    전체적으로 매콤한 편인 줄 모르고 아이를 데려왔는데  그부분에대한 코멘트나 선택사항이 있음 좋을 것 같네요  ',
 '너무추워서따뜻한거시켰어요  찾기가약간힘든데  아담하니 혼밥하기좋아요  이대생들게따뜻한점심이될것같아요  샥슈카는 스프러버여서맛있게먹었어요 다음에는뽀빠이를먹어보고싶어요 ',
 '가격이 최근에 많이 올랐지만  역시나 맛